**Principe de la Méthode** :

**Un hyperchromosome** est une combinaison de paramètres génétiques qui représente une solution potentielle au problème de bin packing. Chaque hyperchromosome est généré aléatoirement avec des valeurs pour les paramètres tels que la taille de la population, le taux de mutation, le taux de croisement, et la taille de la population intermédiaire.

L'algorithme évolutif utilise ces hyperchromosomes pour explorer l'espace des solutions possibles au problème de bin packing. Voici comment cela fonctionne :

**Initialisation de la population** : Une population initiale de hyperchromosomes est générée avec des valeurs aléatoires pour les paramètres génétiques.


**Évolution de la population  **: L'algorithme passe par des générations successives. À chaque génération, la population actuelle d'hyperchromosomes est évaluée en utilisant un algorithme de bin packing pour déterminer la performance de chaque hyperchromosome dans la résolution du problème.


**Opérations génétiques ** : Pendant l'évolution, les hyperchromosomes subissent des opérations génétiques. Certains hyperchromosomes subissent une mutation, ce qui modifie aléatoirement leurs paramètres génétiques. D'autres hyperchromosomes subissent un croisement, où deux hyperchromosomes parents sont combinés pour créer de nouveaux hyperchromosomes enfants.


**Évaluation et sélection **: Une fois que la nouvelle génération d'hyperchromosomes est produite, chaque hyperchromosome est évalué en fonction de sa performance dans la résolution du problème de bin packing. Les meilleurs hyperchromosomes sont sélectionnés pour former la population de la génération suivante, basée sur des critères de sélection tels que le classement des performances.


**Terminaison de l'algorithme **: L'algorithme s'arrête après un nombre fixe d'itérations ou lorsque certains critères de convergence sont atteints. Les hyperchromosomes de la dernière génération sont considérés comme les solutions potentielles au problème de bin packing.


**En résumé**, les hyperchromosomes représentent différentes combinaisons de paramètres génétiques qui sont évaluées par l'algorithme évolutif pour trouver la meilleure solution possible au problème de bin packing. Ces hyperchromosomes guident le processus évolutif en explorant efficacement l'espace des solutions et en sélectionnant les solutions les plus prometteuses pour la génération suivante

In [8]:
import random
import time

class Hyperchromosome:
    def __init__(self, population_size, mutation_rate, crossover_rate, intermediate_population_size):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.intermediate_population_size = intermediate_population_size

    def mutate(self):
        if random.random() < 0.3:
            self.population_size = max(10, self.population_size + random.randint(-10, 10))
        else:
            self.mutation_rate = max(0.01, min(0.1, self.mutation_rate + random.uniform(-0.05, 0.05)))
        self.crossover_rate = max(0.1, min(0.9, self.crossover_rate + random.uniform(-0.1, 0.1)))
        # Update intermediate population size if needed

    def crossover(self, other):
        new_population_size = (self.population_size + other.population_size) // 2
        new_mutation_rate = (self.mutation_rate + other.mutation_rate) / 2
        new_crossover_rate = (self.crossover_rate + other.crossover_rate) / 2
        # Add intermediate population size if needed
        return Hyperchromosome(new_population_size, new_mutation_rate, new_crossover_rate, self.intermediate_population_size)

class GeneticAlgorithm:
    def __init__(self, hyperchromosome, items, capacity):
        self.hyperchromosome = hyperchromosome
        self.items = items
        self.capacity = capacity

    def evolve(self):
        start_time = time.time()

        # Algorithm de bin packing
        bins = []
        current_bin = []
        current_capacity = self.capacity

        # Trie des items dans l'ordre décroissant
        sorted_items = sorted(self.items, reverse=True)

        for item in sorted_items:
            if item <= current_capacity:
                current_bin.append(item)
                current_capacity -= item
            else:
                bins.append(current_bin)
                current_bin = [item]
                current_capacity = self.capacity - item

        bins.append(current_bin)

        num_bins = len(bins)
        end_time = time.time()
        execution_time = end_time - start_time

        return num_bins, execution_time

class HyperheuristicBinPacking:
    def __init__(self, items, capacity):
        self.items = items
        self.capacity = capacity

    def generate_initial_population(self, population_size):
        initial_population = []
        for _ in range(population_size):
            population_size = random.randint(10, 100)
            mutation_rate = random.uniform(0.01, 0.1)
            crossover_rate = random.uniform(0.1, 0.9)
            intermediate_population_size = random.randint(10, 50)  # Set intermediate population size
            hyperchromosome = Hyperchromosome(population_size, mutation_rate, crossover_rate, intermediate_population_size)
            initial_population.append(hyperchromosome)
        return initial_population

    def select_population(self, population, tournament_size):
        selected_population = []
        for _ in range(len(population)):
            tournament = random.sample(population, tournament_size)
            selected_population.append(max(tournament, key=lambda x: x.fitness))
        return selected_population

    def mutate_population(self, population):
        for hyperchromosome in population:
            hyperchromosome.mutate()

    def crossover_population(self, population):
        new_population = []
        random.shuffle(population)
        for i in range(0, len(population), 2):
            if i + 1 < len(population):
                child = population[i].crossover(population[i + 1])
                new_population.append(child)
        return new_population

    def solve(self):
        start_time = time.time()  # Start timing

        initial_population = self.generate_initial_population(population_size=10)

        for generation in range(100):  # Number of iterations of the genetic algorithm

            self.mutate_population(initial_population)
            initial_population.extend(self.crossover_population(initial_population))

            evaluations = {}
            for hyperchromosome in initial_population:
                ga = GeneticAlgorithm(hyperchromosome, self.items, self.capacity)
                num_bins, _ = ga.evolve()  # Don't need execution time here
                evaluations[hyperchromosome] = num_bins

            top_hyperchromosomes = sorted(evaluations.keys(), key=lambda x: evaluations[x])
            initial_population = top_hyperchromosomes[:10]

        best_hyperchromosome = max(evaluations, key=lambda x: evaluations[x])
        num_bins = evaluations[best_hyperchromosome]

        end_time = time.time()  # End timing
        execution_time = end_time - start_time  # Calculate execution time

        return best_hyperchromosome, num_bins, execution_time

# Usage Example
items = [35, 35, 34, 34, 34, 33, 33, 33, 32, 31, 31, 30, 30, 29, 28, 28, 28, 27, 26, 26, 26, 26, 25, 25, 22, 22, 21, 20, 18, 18, 16, 16, 16, 16, 16, 13, 13, 13, 12, 11, 11, 10, 10, 9, 9, 7, 7, 7, 7, 5]
capacity = 50
hyperheuristic_solver = HyperheuristicBinPacking(items, capacity)
best_hyperchromosome, num_bins, execution_time = hyperheuristic_solver.solve()
print("Meilleur hyperchromosome:", best_hyperchromosome.__dict__)
print("Nombre de conteneurs utilisés:", num_bins)
print("Temps d'exécution:", execution_time, "secondes")


Meilleur hyperchromosome: {'population_size': 105, 'mutation_rate': 0.03228574260414272, 'crossover_rate': 0.5526894228649024, 'intermediate_population_size': 17}
Nombre de conteneurs utilisés: 31
Temps d'exécution: 0.018718481063842773 secondes
